In [79]:
import gym
import numpy as np
import gymnasium as gym 


In [80]:
import flappy_bird_gymnasium
import gymnasium as gym

# env = gym.make("FlappyBird-v0", render_mode="human", use_lidar=True)
env = gym.make("FlappyBird-v0",use_lidar=False)
obs, info = env.reset()




In [103]:
obs

array([ 1.        ,  0.21484375,  0.41015625,  1.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  1.        ,  0.4765625 ,
       -0.9       ,  0.5       ])

In [82]:
env.action_space

Discrete(2)

<OrderEnforcing<PassiveEnvChecker<FlappyBirdEnv<FlappyBird-v0>>>>

In [83]:
env.reset()
state,_ = env.reset()
done = False
while not done:
    action = int(np.random.choice([0,1]))
    next_state, reward, done, truncated, _ = env.step(action)
    print(action,reward)

0 0.1
1 0.1
1 0.1
1 0.1
0 0.1
1 0.1
0 0.1
1 0.1
1 0.1
1 0.1
0 0.1
0 0.1
1 0.1
1 0.1
0 0.1
1 0.1
1 0.1
0 0.1
0 0.1
0 0.1
1 0.1
1 0.1
0 0.1
1 0.1
1 0.1
0 0.1
0 0.1
0 0.1
1 0.1
0 -0.5
1 -0.5
0 -0.5
0 -0.5
0 -0.5
0 -0.5
1 -0.5
1 -0.5
1 -0.5
0 -0.5
0 -0.5
1 -0.5
1 -0.5
0 -0.5
0 -0.5
0 -0.5
1 -0.5
0 -0.5
1 -0.5
1 -0.5
0 -1


In [84]:
from torch import nn
import torch

In [85]:
class PolicyNet(nn.Module):
    
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.linear1 = nn.Linear(input_dim,200)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(200,output_dim)
        self.softmax = nn.Softmax(dim=-1)
    
    def forward(self,state):
        ### n
        x = self.linear1(state)
        x = self.relu(x)
        x = self.linear2(x) # n
        x = self.softmax(x) # n
        return x

In [86]:
from torch.distributions import Categorical
import numpy as np
np.bool8 = np.bool_

from torch.optim import AdamW

In [87]:
class Agent:
    
    def __init__(self):
        self.policy_net = PolicyNet(12,2)
        self.optimizer = AdamW(self.policy_net.parameters(),lr=1e-3)
    
    def sample_action(self,state):
        probs = self.policy_net(state) # 4
        if np.random.uniform() < 0.0:
            action = np.random.randint(0,2)
            return action, torch.log(probs[action]+1e-8)
        dist = Categorical(probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        return action.item(),log_prob
    
    def update(self,rewards,log_probs):
        ### 一次游戏时间
        ret = []
        adding = 0
        for r in rewards[::-1]:
            adding = adding * 0.99 + r
            ret.insert(0,adding)
        ret = torch.FloatTensor(ret)
        ret = ret - ret.mean()
        ret = ret / (ret.std()+1e-8)
        
        r_log_probs = []
        for r,log_prob in zip(ret,log_probs):
            r_log_probs.append(-r*log_prob)
        r_log_probs = torch.vstack(r_log_probs)
        
        loss = r_log_probs.sum()
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        return loss

In [88]:
def convert2tensor(state):
    state_arr = torch.FloatTensor(state)
    return state_arr

In [104]:
def train(agent,env):
    success_count = []
    max_size = 1000
    for epoch in range(20000000):
        rewards = []
        log_probs = []
        terminated = False
        success = True
        state,_ = env.reset()
        while not terminated and len(log_probs) < max_size:
            state_arr = convert2tensor(state)
            action, log_prob = agent.sample_action(state_arr)
            next_state, reward, terminated, truncated, _ = env.step(action)
            print('rew',reward)
            state = next_state
            rewards.append(reward)
            log_probs.append(log_prob)
        
        loss = agent.update(rewards,log_probs) 
        success_count.append(success)
        
        
        if (epoch+1) % 10 == 0:
            print(f'epoch: {epoch}, loss: {loss}, rewards: {sum(rewards)}, count: {len(rewards)}')

In [90]:
agent = Agent()



In [105]:
env = gym.make("FlappyBird-v0",use_lidar=False)
train(agent,env)

rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 

rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1


rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 

rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 

rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.

...

rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 

rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 

rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1


rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew -1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0.1
rew 0

KeyboardInterrupt: 

In [92]:
# def sample_action(self,state):
#     probs = self.policy_net(state) # 4
#     if np.random.uniform() < 0.3:
#         action = np.random.randint(0,2)
#         return action, torch.log(probs[action]+1e-8)
#     dist = Categorical(probs)
#     action = dist.sample()
#     log_prob = dist.log_prob(action)
#     return action.item(),log_prob

# # 替换方法
# import types
# agent.sample_action = types.MethodType(sample_action, agent)

In [102]:
import time
def visualize_agent(env, agent, num_episodes=5):
    """
    渲染显示智能体的行动
    """
    env = gym.make('FlappyBird-v0', render_mode='human',use_lidar=False)  # 创建可视化环境
    
    for episode in range(num_episodes):
        state_tuple = env.reset()
        state = state_tuple[0] if isinstance(state_tuple, tuple) else state_tuple
        total_reward = 0
        steps = 0
        done = False
        
        print(f"\nEpisode {episode + 1}")
        
        while not done:
            env.render()  # 渲染当前状态
            
            # 将状态转换为one-hot编码
    
            
            # 使用训练好的策略选择动作
            with torch.no_grad():
                if np.random.random() < 0.0:
                    action = np.random.randint(0, 4)
                else:
                    state_tensor = torch.FloatTensor(state)
                    probs = agent.policy_net(state_tensor)
                    action = probs.argmax().item()  # 使用最可能的动作
            
            # 执行动作
            step_result = env.step(action)
            if len(step_result) == 4:
                next_state, reward, done, _ = step_result
            else:
                next_state, reward, terminated, truncated, _ = step_result
                done = terminated or truncated
            
            total_reward += reward
            steps += 1
            state = next_state
            
            # 添加小延迟使动作更容易观察
            time.sleep(0.01)
        
        print(f"Episode finished after {steps} steps. Total reward: {total_reward}")
    
    env.close()

# 在主程序最后添加：
if __name__ == "__main__":    
    # 训练完成后显示智能体行动
    print("\nVisualizing trained agent behavior...")
    env = gym.make('FlappyBird-v0',render_mode='human',use_lidar=False)
    visualize_agent(env, agent)


Visualizing trained agent behavior...

Episode 1
Episode finished after 7128 steps. Total reward: 880.9000000001037

Episode 2
Episode finished after 27848 steps. Total reward: 3447.8999999983957

Episode 3
Episode finished after 20051 steps. Total reward: 2481.899999999086

Episode 4
Episode finished after 10066 steps. Total reward: 1244.89999999997

Episode 5
Episode finished after 1365 steps. Total reward: 166.89999999999583


In [100]:
env.close()